### ### final2 - 처리 사항
- 변수제거
> groupId,matchId,killplace,(killpoint, rankpoint, winpoint => Weighted),damagedealt,matchduration, numgroups, roadkills, teamkill,vehicle destroys, maxplace

- 이상치 제거
> matchType은 이상치 없음<br>
> 'assists_mean', 'boosts_mean','headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean','longestKill_mean', 'rideDistance_mean','swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean' => quantile 상위 0.005퍼센트 이상치 제거<br>
> 평균값이기 때문에 walkingdistance==0이고 winplaceperc >=0.5 은 제거(핵쟁이일 가능성)

- 데이터 단위 변환
> Walkingdistance, swimdistance, ridingdistance, longestkill – log 변환

- 데이터 변환
> Headshotkills => headshotkillsrate = headshotkill/kill
> matchId, groupId 별로 평균값으로 컬럼 대체
> match 평균 변수 추가
> matchsize 변수 추가

In [37]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm_notebook
import sklearn.preprocessing

In [39]:
# dataframe display 옵션
pd.set_option('display.max_columns' , 50)
pd.set_option('display.width', None)

In [73]:
df_train = pd.read_csv('./data/train_V2.csv', dtype={
                    'assists': 'int16',
                    'boosts': 'int16',
                    'DBNOs': 'int16',
                    'headshotKills': 'int16',
                    'heals': 'int16',
                    'kills': 'int16',
                    'killStreaks': 'int16',
                    'longestKill': 'float32',
                    'revives': 'int16',
                    'rideDistance': 'float32',
                    'swimDistance': 'float32',
                    'vehicleDestroys': 'int16',
                    'walkDistance': 'float32',
                    'weaponsAquired': 'int16',
                    'winPlacePerc': 'float32'
                })

In [74]:
# 필요 없는 칼럼 제거
del_list = ['maxPlace','damageDealt','killPlace', 'killPoints',
            'matchDuration','numGroups', 'rankPoints','roadKills','teamKills','vehicleDestroys','winPoints']
for col in del_list :
    del df_train[col]

In [42]:
# ## 메모리 절약
# def reduce_mem_usage(df):
#     """ iterate through all the columns of a dataframe and modify the data type
#         to reduce memory usage.        
#     """
#     #start_mem = df.memory_usage().sum() / 1024**2
#     #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
#     for col in df.columns:
#         col_type = df[col].dtype

#         if col_type != object:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)  
#             else:
#                 if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)

#     #end_mem = df.memory_usage().sum() / 1024**2
#     #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
#     #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

#     return df

In [43]:
# df_train = reduce_mem_usage(df_train)

In [75]:
df_train_mean = df_train.groupby(['matchId','groupId']).mean()
df_train_size = df_train.groupby(['matchId']).size().reset_index(name='match_size')
df_train_match_mean = df_train.groupby(['matchId']).mean()

In [76]:
df_train = pd.merge(df_train, df_train_mean, suffixes=["","_mean"], how = 'left', on=['matchId','groupId'])

In [77]:
del df_train_mean

In [78]:
df_train = pd.merge(df_train,df_train_match_mean, suffixes=["","_match_mean"], how="left", on=['matchId'])

In [79]:
del df_train_match_mean

In [80]:
df_train = pd.merge(df_train, df_train_size, how='left', on=['matchId'])

In [81]:
del df_train_size

In [82]:
df_train_cvt = df_train.copy()

In [83]:
# 회귀분석을 하기 위해 문자열을 범주값으로 변환
matchTyp = {'squad-fpp': 3, 'duo': 2, 'solo-fpp': 1, 'squad': 3, 'duo-fpp': 2, 'solo': 1,
       'normal-squad-fpp': 3, 'crashfpp': 4, 'flaretpp': 4, 'normal-solo-fpp': 1,
       'flarefpp': 4, 'normal-duo-fpp': 2, 'normal-duo': 2, 'normal-squad': 3,
       'crashtpp': 4, 'normal-solo': 1 }

df_train_cvt['matchType'] = df_train_cvt['matchType'].replace(matchTyp)

In [84]:
#필요 없는 칼럼 제거
del_list = ['assists', 'boosts', 'DBNOs',
       'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
       'revives', 'rideDistance', 'swimDistance', 'walkDistance',
       'weaponsAcquired','winPlacePerc_match_mean','winPlacePerc_mean']
for col in del_list :
    del df_train_cvt[col]

In [85]:
# null값 확인 및 
# inplace=False로 하면 기존 혹은 새로운 변수에 할당해야 하고, inplace = True 하면 해당변수에 적용됨
# na가 포함된 행을 제거하는 것은 데이터 소실이 크기 때문에 inplace=False가 default
df_train_cvt.dropna(inplace = True)
print(df_train_cvt.isnull().any().any())

False


### solo

In [100]:
solo_train_df = df_train_cvt[df_train_cvt['matchType']==1]

In [101]:
del solo_train_df['DBNOs_mean']
del solo_train_df['revives_mean']
del solo_train_df['DBNOs_match_mean']
del solo_train_df['revives_match_mean']

In [102]:
# solo
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean']
quantile_dic = {}
for col in quantile_list:
    tmp_np = np.quantile(solo_train_df[col],0.995)
    quantile_dic[col] = tmp_np
print(quantile_dic)

{'assists_mean': 1.0, 'boosts_mean': 9.0, 'headshotKills_mean': 4.0, 'heals_mean': 14.0, 'kills_mean': 9.0, 'killStreaks_mean': 2.0, 'longestKill_mean': 267.0, 'rideDistance_mean': 7901.0, 'swimDistance_mean': 218.8000030517578, 'walkDistance_mean': 4612.444999999949, 'weaponsAcquired_mean': 12.0}


In [103]:
# 이상치 제거 1
for col in quantile_dic:
    solo_train_df = solo_train_df[solo_train_df[col] <= quantile_dic[col]]

In [104]:
# 이상치 제거 2
solo_train_df.drop(index=solo_train_df[(solo_train_df['walkDistance_mean']==0)&(solo_train_df['winPlacePerc']>=0.5)].index,inplace=True)

In [105]:
# data 단위 변환
log_transform = ['longestKill_mean','rideDistance_mean','swimDistance_mean', 'walkDistance_mean']
for col in log_transform :
    solo_train_df[col+'_log'] = np.log(solo_train_df[col]+1)

In [106]:
# data 변환 - headshotkills
solo_train_df['headshotKillrate_mean'] = solo_train_df['headshotKills_mean']/solo_train_df['kills_mean']
solo_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)

In [107]:
solo_train_df.isna().any().any()

False

In [108]:
# winPlacePerc 컬럼위치 변경
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
solo_train_df = change_column_order(solo_train_df,'winPlacePerc',len(list(solo_train_df.columns)))

In [109]:
solo_train_df.describe()

,matchType,assists_mean,boosts_mean,headshotKills_mean,heals_mean,kills_mean,killStreaks_mean,longestKill_mean,rideDistance_mean,swimDistance_mean,walkDistance_mean,weaponsAcquired_mean,assists_match_mean,boosts_match_mean,headshotKills_match_mean,heals_match_mean,kills_match_mean,killStreaks_match_mean,longestKill_match_mean,rideDistance_match_mean,swimDistance_match_mean,walkDistance_match_mean,weaponsAcquired_match_mean,match_size,longestKill_mean_log,rideDistance_mean_log,swimDistance_mean_log,walkDistance_mean_log,headshotKillrate_mean,winPlacePerc
count,693939.0,693939.000000,693939.00000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000,693939.000000
mean,1.0,0.049842,0.95790,0.217153,0.858751,0.843354,0.456451,18.780785,388.596466,3.039263,895.636902,3.470164,0.057297,1.058939,0.245022,1.004489,0.920381,0.473272,21.394835,452.601532,5.028447,951.964722,3.548089,94.072436,1.399889,1.338985,0.193060,5.736931,0.109436,0.473902
std,0.0,0.216469,1.63203,0.542003,1.857281,1.316268,0.534698,38.659321,1115.795654,17.443048,992.350464,2.262388,0.032311,0.201712,0.062007,0.342490,0.124546,0.063579,5.415330,490.404449,5.455434,230.548248,0.504960,9.552324,1.755477,2.835694,0.843354,1.912825,0.270982,0.293855
min,1.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,95.699997,2.000000,0.040816,0.938776,0.212121,0.762887,0.905263,0.451613,17.979870,71.087006,1.618021,791.148926,3.164835,94.000000,0.000000,0.000000,0.000000,4.571613,0.000000,0.218800
50%,1.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,459.200012,3.000000,0.053191,1.052632,0.244898,0.949495,0.936842,0.478723,20.739262,135.611710,3.595186,879.000732,3.434343,96.000000,0.000000,0.000000,0.000000,6.131661,0.000000,0.466700
75%,1.0,0.000000,1.00000,0.000000,1.000000,1.000000,1.000000,18.900000,0.000000,0.000000,1494.000000,5.000000,0.072917,1.178947,0.278351,1.214286,0.958333,0.505155,24.304344,878.026062,6.570538,1143.664795,3.926316,97.000000,2.990720,0.000000,0.000000,7.309882,0.000000,0.725300
max,1.0,1.000000,9.00000,4.000000,14.000000,9.000000,2.000000,267.000000,7901.000000,218.800003,4612.000000,12.000000,4.659574,4.866667,6.086957,6.555556,14.702128,2.695652,181.509613,3479.573730,53.829189,2946.185791,32.961538,100.000000,5.590987,8.974872,5.392718,8.436634,1.000000,1.000000


In [110]:
solo_train_df.to_csv('./data/final2_solo_Data.csv',sep=',', encoding='utf-8', index=False)

### duo

In [115]:
duo_train_df = df_train_cvt[df_train_cvt['matchType']==2]

#  duo 리스트
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean']
quantile_dic = {}
for col in quantile_list:
    tmp_np = np.quantile(duo_train_df[col],0.995)
    quantile_dic[col] = tmp_np
print(quantile_dic)

# 이상치 제거 1
for col in quantile_dic:
    duo_train_df = duo_train_df[duo_train_df[col] <= quantile_dic[col]]
    
# 이상치 제거 2
duo_train_df.drop(index=duo_train_df[(duo_train_df['walkDistance_mean']==0)&(duo_train_df['winPlacePerc']>=0.5)].index,inplace=True)

# data 단위 변환
log_transform = ['longestKill_mean','rideDistance_mean','swimDistance_mean', 'walkDistance_mean']
for col in log_transform :
    duo_train_df[col+'_log'] = np.log(duo_train_df[col]+1)
    
# data 변환 - headshotkills
duo_train_df['headshotKillrate_mean'] = duo_train_df['headshotKills_mean']/duo_train_df['kills_mean']
duo_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)

# winPlacePerc 컬럼위치 변경
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
duo_train_df = change_column_order(duo_train_df,'winPlacePerc',len(list(duo_train_df.columns)))

duo_train_df.to_csv('./data/final2_duo_Data.csv',sep=',', encoding='utf-8', index=False)

{'assists_mean': 2.5, 'boosts_mean': 7.5, 'headshotKills_mean': 2.5, 'heals_mean': 12.5, 'kills_mean': 6.5, 'killStreaks_mean': 2.0, 'longestKill_mean': 224.4499969482422, 'rideDistance_mean': 7723.5, 'swimDistance_mean': 177.9577603149428, 'walkDistance_mean': 4698.0, 'weaponsAcquired_mean': 9.5, 'DBNOs_mean': 4.0, 'revives_mean': 1.5}


### squad

In [116]:
squad_train_df = df_train_cvt[df_train_cvt['matchType']==3]

#  squad 리스트
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean']
quantile_dic = {}
for col in quantile_list:
    tmp_np = np.quantile(squad_train_df[col],0.995)
    quantile_dic[col] = tmp_np
print(quantile_dic)

# 이상치 제거 1
for col in quantile_dic:
    squad_train_df = squad_train_df[squad_train_df[col] <= quantile_dic[col]]
    
# 이상치 제거 2
squad_train_df.drop(index=squad_train_df[(squad_train_df['walkDistance_mean']==0)&(squad_train_df['winPlacePerc']>=0.5)].index,inplace=True)

# data 단위 변환
log_transform = ['longestKill_mean','rideDistance_mean','swimDistance_mean', 'walkDistance_mean']
for col in log_transform :
    squad_train_df[col+'_log'] = np.log(squad_train_df[col]+1)
    
# data 변환 - headshotkills
squad_train_df['headshotKillrate_mean'] = squad_train_df['headshotKills_mean']/squad_train_df['kills_mean']
squad_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)

# winPlacePerc 컬럼위치 변경
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
squad_train_df = change_column_order(squad_train_df,'winPlacePerc',len(list(squad_train_df.columns)))

squad_train_df.to_csv('./data/final2_squad_Data.csv',sep=',', encoding='utf-8', index=False)

{'assists_mean': 2.5, 'boosts_mean': 6.25, 'headshotKills_mean': 2.0, 'heals_mean': 11.5, 'kills_mean': 6.0, 'killStreaks_mean': 2.0, 'longestKill_mean': 206.5, 'rideDistance_mean': 7623.0, 'swimDistance_mean': 149.8000030517578, 'walkDistance_mean': 4663.0, 'weaponsAcquired_mean': 9.5, 'DBNOs_mean': 4.666666666666667, 'revives_mean': 1.6666666666666667}


### other

In [117]:
other_train_df = df_train_cvt[df_train_cvt['matchType']==4]

#  other 리스트
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean']
quantile_dic = {}
for col in quantile_list:
    tmp_np = np.quantile(other_train_df[col],0.995)
    quantile_dic[col] = tmp_np
print(quantile_dic)

# 이상치 제거 1
for col in quantile_dic:
    other_train_df = other_train_df[other_train_df[col] <= quantile_dic[col]]
    
# 이상치 제거 2
other_train_df.drop(index=other_train_df[(other_train_df['walkDistance_mean']==0)&(other_train_df['winPlacePerc']>=0.5)].index,inplace=True)

# data 단위 변환
log_transform = ['longestKill_mean','rideDistance_mean','swimDistance_mean', 'walkDistance_mean']
for col in log_transform :
    other_train_df[col+'_log'] = np.log(other_train_df[col]+1)
    
# data 변환 - headshotkills
other_train_df['headshotKillrate_mean'] = other_train_df['headshotKills_mean']/other_train_df['kills_mean']
other_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)

# winPlacePerc 컬럼위치 변경
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
other_train_df = change_column_order(other_train_df,'winPlacePerc',len(list(other_train_df.columns)))

other_train_df.to_csv('./data/final2_other_Data.csv',sep=',', encoding='utf-8', index=False)

{'assists_mean': 2.25, 'boosts_mean': 6.25, 'headshotKills_mean': 1.5, 'heals_mean': 12.0, 'kills_mean': 6.0, 'killStreaks_mean': 2.5, 'longestKill_mean': 168.20999145507812, 'rideDistance_mean': 8028.33349609375, 'swimDistance_mean': 124.5250015258789, 'walkDistance_mean': 4541.60009765625, 'weaponsAcquired_mean': 9.0, 'DBNOs_mean': 4.0, 'revives_mean': 1.5}


In [119]:
other_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9472 entries, 1093 to 4442766
Data columns (total 37 columns):
Id                            9472 non-null object
groupId                       9472 non-null object
matchId                       9472 non-null object
matchType                     9472 non-null int64
assists_mean                  9472 non-null float64
boosts_mean                   9472 non-null float64
DBNOs_mean                    9472 non-null float64
headshotKills_mean            9472 non-null float64
heals_mean                    9472 non-null float64
kills_mean                    9472 non-null float64
killStreaks_mean              9472 non-null float64
longestKill_mean              9472 non-null float32
revives_mean                  9472 non-null float64
rideDistance_mean             9472 non-null float32
swimDistance_mean             9472 non-null float32
walkDistance_mean             9472 non-null float32
weaponsAcquired_mean          9472 non-null float64
assists_